Importing dependencies


In [3]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data Pre-proessing& Data collection

In [12]:
#loading data from csv file
movie_data= pd.read_csv('/content/movies.csv')
#number of rows and columns
print(movie_data.shape)
print(movie_data.head())

(4803, 24)
   index     budget                                    genres  \
0      0  237000000  Action Adventure Fantasy Science Fiction   
1      1  300000000                  Adventure Fantasy Action   
2      2  245000000                    Action Adventure Crime   
3      3  250000000               Action Crime Drama Thriller   
4      4  260000000          Action Adventure Science Fiction   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  culture clash future space war space colony so...                en   
1  ocean drug abuse exotic island east india trad...                en   
2         spy based o

In [13]:
featured_features=['genres','keywords','tagline','cast','director']
print(featured_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [14]:
for feature in featured_features:
    movie_data[feature] = movie_data[feature].fillna('')
combined_features = movie_data['genres'] + ' ' + movie_data['keywords'] + ' ' + movie_data['tagline'] + ' ' + movie_data['cast'] + ' ' + movie_data['director']


In [16]:
#converting data into feature vectors
vectorizer = TfidfVectorizer(stop_words='english')
feat_vector = vectorizer.fit_transform(combined_features)


In [17]:
#getting similarity scores using cosine similarity
similar=cosine_similarity(feat_vector)
print(similar)
print(similar.shape)

[[1.         0.04720973 0.03891121 ... 0.         0.         0.        ]
 [0.04720973 1.         0.021542   ... 0.01267122 0.         0.        ]
 [0.03891121 0.021542   1.         ... 0.         0.05626942 0.        ]
 ...
 [0.         0.01267122 0.         ... 1.         0.         0.02730605]
 [0.         0.         0.05626942 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02730605 0.         1.        ]]
(4803, 4803)


Movie Recommendation Function



In [18]:
def recommend_movies(movie_name, num_recommendations=10):
    if movie_name not in movie_data['title'].values:
        return "Movie not found in the dataset."
    list_all_titles = movie_data['title'].tolist()

    # finding close match for the input movie
    close_matches = difflib.get_close_matches(movie_name, list_all_titles)
    if not close_matches:
        return "No matching movies found. Please try another title."

    # find index of movie with title
    close_match = close_matches[0]
    index_movie = movie_data[movie_data.title == close_match].index[0]

    # getting list of similar movies
    similar_score = list(enumerate(similar[index_movie]))
    sorted_similar_movies = sorted(similar_score, key=lambda x: x[1], reverse=True)[1:num_recommendations+1]

    # print movies based on index
    print("Suggested Movies for you:\n")
    recommendations = []
    for i, (index, score) in enumerate(sorted_similar_movies, 1):
        title_from_index = movie_data.iloc[index]['title']
        recommendations.append(f"{i}. {title_from_index}")

    return "\n".join(recommendations)


User Input and Recommendation Disaplay

In [19]:
movie_name = input("Enter a movie name: ")
print(recommend_movies(movie_name, num_recommendations=10))


Enter a movie name: King Kong
Suggested Movies for you:

1. The Last Time I Committed Suicide
2. Redbelt
3. America's Sweethearts
4. What Just Happened
5. Ed Wood
6. Unfinished Business
7. The Pianist
8. Big Fat Liar
9. Bowfinger
10. Tropic Thunder
